# How is an image formed?

```{admonition} Use the Live Code feature on this page using the <i class="fas fa-rocket"></i> icon on top.
:class: tip
Live Code feature works on this page.
```

<div>
<iframe src="https://slides.com/naresh-ub/cvip-lec-2-3/embed" width="100%" height="500" title="Test Title" scrolling="no" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe>
</div>

In this lecture, we shall understand the Math and Geometry behind Image Formation. In the [previous lecture](./lecture-1.ipynb), we learned that images are stored/read as `numpy arrays`. In this lecture we understand the way in which a 3D object is captured on a 2D image plane. In fact, we will see that a `camera is nothing but a function that maps a 3D point in real-world onto a 2D point on the image plane`.

Let's get started with an object that we want to capture. For this lecture, let's choose an object with regular shape.

In [23]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display

# === Toggle Render Mode: 'vscode' or 'iframe'
render_mode = 'iframe'  # ← change to 'iframe' for notebook inline view

if render_mode == 'vscode':
    pio.renderers.default = 'vscode'
else:
    pio.renderers.default = 'iframe'

# === Cube Parameters ===
cube_size = 2
cube_center = np.array([1.5, 1.5, 1.5])  # ← updated center

# === Cube Vertices ===
half = cube_size / 2
local_vertices = np.array([
    [-half, -half, -half], [ half, -half, -half],
    [ half,  half, -half], [-half,  half, -half],
    [-half, -half,  half], [ half, -half,  half],
    [ half,  half,  half], [-half,  half,  half],
])
vertices = local_vertices + cube_center
x, y, z = vertices.T

# === Cube Faces ===
faces = np.array([
    [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
    [0, 1, 5], [0, 5, 4], [1, 2, 6], [1, 6, 5],
    [2, 3, 7], [2, 7, 6], [3, 0, 4], [3, 4, 7]
])
i, j, k = faces.T

# === Face Colors ===
face_colors = ['lightgray'] * len(faces)
highlight_faces = {'orange': [10, 11], 'green': [6, 7]}
for idx in highlight_faces['orange']:
    face_colors[idx] = 'orange'
for idx in highlight_faces['green']:
    face_colors[idx] = 'green'

# === Cube Mesh ===
object_mesh = go.Mesh3d(
    x=x, y=y, z=z,
    i=i, j=j, k=k,
    facecolor=face_colors,
    opacity=0.5,
    name='Cube'
)

# === Vertex Labels ===
corner_labels = go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers+text',
    marker=dict(size=3, color='red'),
    text=[f'v{i}' for i in range(8)],
    textposition='top center',
    name='Vertices'
)

# === Global Axes with Labels (origin-centered) ===
axis_len = 4
world_axes = [
    go.Scatter3d(x=[-axis_len, axis_len], y=[0, 0], z=[0, 0],
                 mode='lines', line=dict(color='red', width=5), name='X-axis'),
    go.Scatter3d(x=[0, 0], y=[-axis_len, axis_len], z=[0, 0],
                 mode='lines', line=dict(color='green', width=5), name='Y-axis'),
    go.Scatter3d(x=[0, 0], y=[0, 0], z=[-1, axis_len],
                 mode='lines', line=dict(color='blue', width=5), name='Z-axis'),
    go.Scatter3d(x=[axis_len], y=[0], z=[0],
                 mode='text', text=["X"], textposition='top right',
                 textfont=dict(size=14, color='red'), showlegend=False),
    go.Scatter3d(x=[0], y=[axis_len], z=[0],
                 mode='text', text=["Y"], textposition='top right',
                 textfont=dict(size=14, color='green'), showlegend=False),
    go.Scatter3d(x=[0], y=[0], z=[axis_len],
                 mode='text', text=["Z"], textposition='top right',
                 textfont=dict(size=14, color='blue'), showlegend=False),
]

# === Plot Figure ===
fig = go.Figure(data=[object_mesh, corner_labels, *world_axes])
fig.update_layout(
    title='Floating Cube with Global Axes',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        camera=dict(eye=dict(x=0.8, y=0.8, z=0.8))  # Zoomed-out camera view
    ),
    margin=dict(l=0, r=0, t=40, b=40),
    showlegend=True
)

# === Save and Display ===
if render_mode == 'iframe':
    folder = "iframe_figures"
    filename = "cube_with_axes.html"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, filename)
    fig.write_html(file_path, include_plotlyjs='cdn', full_html=True)
    display(IFrame(src=file_path, width='100%', height='400'))
else:
    fig.show()

In the above 3D Plot (using Plotly), we are able to visualize a 3D Cube. Let's say we want to study how a 3D object (like the cube above) is captured onto a 2D image plane. We study this using _Perspective Projection and the Pinhole Camera model_.

In [3]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display

# === Toggle Render Mode: 'vscode' or 'iframe'
render_mode = 'iframe'  # ← change to 'iframe' to render inline iframe

if render_mode == 'vscode':
    pio.renderers.default = 'vscode'
else:
    pio.renderers.default = 'iframe'

# === Parameters ===
cube_size = 2
cube_center = np.array([0, 0, 1.5])
pinhole = np.array([-6, 0, 0])
image_plane_x = -8

# === Cube vertices ===
half = cube_size / 2
local_vertices = np.array([
    [-half, -half, -half], [ half, -half, -half],
    [ half,  half, -half], [-half,  half, -half],
    [-half, -half,  half], [ half, -half,  half],
    [ half,  half,  half], [-half,  half,  half],
])
vertices = local_vertices + cube_center
x, y, z = vertices.T

# === Faces
faces = np.array([
    [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
    [0, 1, 5], [0, 5, 4], [1, 2, 6], [1, 6, 5],
    [2, 3, 7], [2, 7, 6], [3, 0, 4], [3, 4, 7]
])
i, j, k = faces.T

# === Face colors
face_colors = ['lightgray'] * len(faces)
highlight_faces = {'orange': [10, 11], 'green': [6, 7]}
for idx in highlight_faces['orange']:
    face_colors[idx] = 'orange'
for idx in highlight_faces['green']:
    face_colors[idx] = 'green'

# === Perspective projection
def project_point(P, C, image_plane_x):
    t = (image_plane_x - C[0]) / (P[0] - C[0])
    return C + t * (P - C)

projected_points = np.array([project_point(P, pinhole, image_plane_x) for P in vertices])
proj_x, proj_y, proj_z = projected_points.T

# === Plot elements
image_mesh = go.Mesh3d(x=proj_x, y=proj_y, z=proj_z, i=i, j=j, k=k,
                       facecolor=face_colors, opacity=0.6, name='Projected Image')
object_mesh = go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k,
                        facecolor=face_colors, opacity=0.4, name='Block')
yy, zz = np.meshgrid(np.linspace(-3, 3, 2), np.linspace(-3, 3, 2))
xx = np.full_like(yy, image_plane_x)
image_plane = go.Surface(x=xx, y=yy, z=zz, showscale=False, opacity=0.3,
                         name='Image Plane', colorscale='gray')

# === Rays
rays = [go.Scatter3d(
    x=[vertices[i][0], pinhole[0], projected_points[i][0]],
    y=[vertices[i][1], pinhole[1], projected_points[i][1]],
    z=[vertices[i][2], pinhole[2], projected_points[i][2]],
    mode='lines',
    line=dict(color='gray', width=2, dash='dot'),
    showlegend=False
) for i in range(8)]

# === Labels
corner_labels = go.Scatter3d(x=x, y=y, z=z, mode='markers+text',
    marker=dict(size=2, color='red'), text=[f'v{i}' for i in range(8)],
    textposition='top center', name='Vertices')
image_points = go.Scatter3d(
    x=proj_x, y=proj_y, z=proj_z,
    mode='markers+text',
    marker=dict(size=2, color='black'),
    text=[f'p{i}' for i in range(8)],
    textposition='bottom center',
    textfont=dict(size=8),  # ← Smaller label font
    name='Projected Corners'
)

# === Pinhole, Axis
pinhole_marker = go.Scatter3d(x=[pinhole[0]], y=[pinhole[1]], z=[pinhole[2]],
    mode='markers+text', marker=dict(size=4, color='purple'),
    text=["Pinhole"], textposition='bottom center', showlegend=False)
optical_axis = go.Scatter3d(x=[image_plane_x - 2, cube_center[0] + 3],
    y=[0, 0], z=[0, 0], mode='lines',
    line=dict(color='black', width=4, dash='dash'), name='Optical Axis')

# === World and Camera Axes
axis_len = 2
world_axes = [
    go.Scatter3d(x=[cube_center[0], cube_center[0] + axis_len], y=[cube_center[1], cube_center[1]], z=[cube_center[2], cube_center[2]], mode='lines', line=dict(color='red', width=5), name='World X'),
    go.Scatter3d(x=[cube_center[0], cube_center[0]], y=[cube_center[1], cube_center[1] + axis_len], z=[cube_center[2], cube_center[2]], mode='lines', line=dict(color='green', width=5), name='World Y'),
    go.Scatter3d(x=[cube_center[0], cube_center[0]], y=[cube_center[1], cube_center[1]], z=[cube_center[2], cube_center[2] + axis_len], mode='lines', line=dict(color='blue', width=5), name='World Z')
]
camera_axes = [
    go.Scatter3d(x=[pinhole[0], pinhole[0] + axis_len], y=[pinhole[1], pinhole[1]], z=[pinhole[2], pinhole[2]], mode='lines', line=dict(color='red', width=5), name="X′ (Camera)"),
    go.Scatter3d(x=[pinhole[0], pinhole[0]], y=[pinhole[1], pinhole[1] + axis_len], z=[pinhole[2], pinhole[2]], mode='lines', line=dict(color='green', width=5), name="Y′ (Camera)"),
    go.Scatter3d(x=[pinhole[0], pinhole[0]], y=[pinhole[1], pinhole[1]], z=[pinhole[2], pinhole[2] + axis_len], mode='lines', line=dict(color='blue', width=5), name="Z′ (Camera)")
]

# === Final Figure ===
fig = go.Figure(data=[
    object_mesh, image_mesh, image_plane,
    corner_labels, image_points, pinhole_marker, optical_axis,
    *world_axes, *camera_axes, *rays
])
fig.update_layout(
    title='Perspective Projection of a Cube onto an Image Plane',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        camera=dict(
            eye=dict(x=0.8, y=0.8, z=0.8)  # ← Controls default zoom level
        )
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=0, r=0, t=40, b=40),
    showlegend=True
)

# === Save and Show ===
if render_mode == 'iframe':
    folder = "iframe_figures"
    filename = "cube_projection.html"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, filename)
    fig.write_html(file_path, include_plotlyjs='cdn', full_html=True)
    display(IFrame(src=file_path, width='100%', height='500'))
else:
    fig.show()

In [25]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display

# === Toggle Render Mode: 'vscode' or 'iframe'
render_mode = 'iframe'  # ← change to 'vscode' if needed

if render_mode == 'vscode':
    pio.renderers.default = 'vscode'
else:
    pio.renderers.default = 'iframe'

# === Parameters
pinhole = np.array([-6, 0, 0])
image_plane_x = -8

# === Create 3D sphere
phi, theta = np.mgrid[0:np.pi:20j, 0:2*np.pi:40j]
r = 1.5
x = r * np.sin(phi) * np.cos(theta) + 1.5  # Shifted forward
y = r * np.sin(phi) * np.sin(theta)
z = r * np.cos(phi)

sphere_points = np.stack([x.ravel(), y.ravel(), z.ravel()], axis=1)

# === Perspective projection
def project_point(P, C, x_plane):
    t = (x_plane - C[0]) / (P[0] - C[0])
    return C + t * (P - C)

projected_points = np.array([project_point(p, pinhole, image_plane_x) for p in sphere_points])
proj_x = projected_points[:, 0].reshape(phi.shape)
proj_y = projected_points[:, 1].reshape(phi.shape)
proj_z = projected_points[:, 2].reshape(phi.shape)

# === Surfaces
sphere_surface = go.Surface(x=x, y=y, z=z, colorscale='Blues', opacity=0.7, showscale=False, name='Sphere')
projected_surface = go.Surface(x=proj_x, y=proj_y, z=proj_z, colorscale='Blues', opacity=0.6, showscale=False, name='Projected Sphere')

# === Image plane
plane_size = 3.5
yy, zz = np.meshgrid(np.linspace(-plane_size, plane_size, 2), np.linspace(-plane_size, plane_size, 2))
xx = np.full_like(yy, image_plane_x)
image_plane = go.Surface(x=xx, y=yy, z=zz, showscale=False, opacity=0.3, name='Image Plane', colorscale='gray')

# === Pinhole and axis
pinhole_marker = go.Scatter3d(x=[pinhole[0]], y=[pinhole[1]], z=[pinhole[2]],
                              mode='markers+text', marker=dict(size=4, color='purple'),
                              text=["Pinhole"], name='Pinhole')
optical_axis = go.Scatter3d(x=[image_plane_x - 2, 3], y=[0, 0], z=[0, 0],
                            mode='lines', line=dict(color='black', width=4, dash='dash'), name='Optical Axis')

# === Axes
center = np.array([1.5, 0, 0])
def axis_line(origin, direction, color, name):
    return go.Scatter3d(x=[origin[0], origin[0] + direction[0]],
                        y=[origin[1], origin[1] + direction[1]],
                        z=[origin[2], origin[2] + direction[2]],
                        mode='lines', line=dict(color=color, width=5), name=name)

world_axes = [
    axis_line(center, [2, 0, 0], 'red', 'World X'),
    axis_line(center, [0, 2, 0], 'green', 'World Y'),
    axis_line(center, [0, 0, 2], 'blue', 'World Z')
]
cam_axes = [
    axis_line(pinhole, [2, 0, 0], 'red', "X′ (Camera)"),
    axis_line(pinhole, [0, 2, 0], 'green', "Y′ (Camera)"),
    axis_line(pinhole, [0, 0, 2], 'blue', "Z′ (Camera)")
]

# === Final figure
fig = go.Figure(data=[
    sphere_surface, projected_surface, image_plane,
    pinhole_marker, optical_axis,
    *world_axes, *cam_axes
])
fig.update_layout(
    title='Perspective Projection of a 3D Sphere',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        camera=dict(eye=dict(x=1.1, y=1.1, z=1.1))  # Zoom level
    ),
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5),
    margin=dict(l=0, r=0, t=40, b=40)
)

# === Save and Show
if render_mode == 'iframe':
    folder = "iframe_figures"
    filename = "sphere_projection.html"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, filename)
    fig.write_html(file_path, include_plotlyjs='cdn', full_html=True)
    display(IFrame(src=file_path, width='100%', height='500'))
else:
    fig.show()

In [26]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display

pio.renderers.default = 'iframe'

# Triangle Prism (upright and perpendicular to camera)
triangle_vertices = np.array([
    [0, -1, -1], [0, 1, -1], [0, 0, 1],      # front triangle
    [2, -1, -1], [2, 1, -1], [2, 0, 1]       # back triangle
])
tri_faces = [
    [0, 1, 2], [3, 5, 4],
    [0, 1, 4], [0, 4, 3],
    [1, 2, 5], [1, 5, 4],
    [2, 0, 3], [2, 3, 5]
]

image_plane_x = -8

def project_points(points, pinhole, x_plane):
    projected = []
    for p in points:
        t = (x_plane - pinhole[0]) / (p[0] - pinhole[0] + 1e-6)
        proj = pinhole + t * (p - pinhole)
        projected.append(proj)
    return np.array(projected)

def axis_line(origin, direction, color, name):
    return go.Scatter3d(
        x=[origin[0], origin[0] + direction[0]],
        y=[origin[1], origin[1] + direction[1]],
        z=[origin[2], origin[2] + direction[2]],
        mode='lines', line=dict(color=color, width=5), name=name
    )

def triangle_mesh(verts, face_indices, color, name, opacity=0.7):
    x, y, z = verts[:,0], verts[:,1], verts[:,2]
    i, j, k = zip(*face_indices)
    return go.Mesh3d(
        x=x, y=y, z=z, i=i, j=j, k=k,
        color=color, opacity=opacity, name=name
    )

def make_frame(pinhole_x):
    pinhole = np.array([pinhole_x, 0, 0])
    projected = project_points(triangle_vertices, pinhole, image_plane_x)
    proj_mesh = triangle_mesh(projected, tri_faces, 'green', 'Projected Triangle', opacity=0.6)
    pinhole_marker = go.Scatter3d(
        x=[pinhole[0]], y=[pinhole[1]], z=[pinhole[2]],
        mode='markers+text', marker=dict(size=4, color='purple'),
        text=["Pinhole"], name='Pinhole'
    )
    cam_axes = [
        axis_line(pinhole, [2, 0, 0], 'red', "X′ (Camera)"),
        axis_line(pinhole, [0, 2, 0], 'green', "Y′ (Camera)"),
        axis_line(pinhole, [0, 0, 2], 'blue', "Z′ (Camera)")
    ]
    return [proj_mesh, pinhole_marker, *cam_axes]

object_mesh = triangle_mesh(triangle_vertices, tri_faces, 'orange', 'Triangle Prism')
center = triangle_vertices.mean(axis=0)
world_axes = [
    axis_line(center, [2, 0, 0], 'red', 'World X'),
    axis_line(center, [0, 2, 0], 'green', 'World Y'),
    axis_line(center, [0, 0, 2], 'blue', 'World Z')
]

plane_size = 3.5
yy, zz = np.meshgrid(np.linspace(-plane_size, plane_size, 2), np.linspace(-plane_size, plane_size, 2))
xx = np.full_like(yy, image_plane_x)
image_plane = go.Surface(x=xx, y=yy, z=zz, showscale=False, opacity=0.3, name='Image Plane', colorscale='gray')

optical_axis = go.Scatter3d(x=[image_plane_x - 2, 3], y=[0, 0], z=[0, 0],
                            mode='lines', line=dict(color='black', width=4, dash='dash'), name='Optical Axis')

# === Animation frames and slider
initial_pinhole_x = -6
initial_frame = make_frame(initial_pinhole_x)
frames = []
slider_steps = []
slider_range = np.linspace(-7, -2, 30)
for px in slider_range:
    frame_data = [object_mesh, image_plane, optical_axis, *world_axes, *make_frame(px)]
    frames.append(go.Frame(data=frame_data, name=str(px)))
    slider_steps.append(dict(method="animate", args=[[str(px)], {"frame": {"duration": 50, "redraw": True}, "mode": "immediate"}],
                             label=f"{px:.1f}"))

fig = go.Figure(
    data=[object_mesh, image_plane, optical_axis, *world_axes, *initial_frame],
    frames=frames
)

fig.update_layout(
    title='Perspective Projection of a 3D Triangle Prism with Movable Pinhole',
    updatemenus=[dict(type="buttons", showactive=False,
                      x=0.05, y=1.05,
                      buttons=[dict(label="Play", method="animate",
                                    args=[None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}])])],
    sliders=[dict(
        steps=slider_steps,
        active=4,
        x=0.5, xanchor="center",
        y=0, yanchor="bottom",
        pad=dict(t=0, b=0),
        currentvalue=dict(prefix="Pinhole X: ", xanchor="center", font=dict(size=14))
    )],
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        camera=dict(eye=dict(x=1.0, y=1.0, z=1.0))
    ),
    showlegend=False,
    margin=dict(l=0, r=0, t=50, b=50)
)

# === Save and display in iframe
folder = "iframe_figures"
filename = "triangle_prism_slider.html"
os.makedirs(folder, exist_ok=True)
file_path = os.path.join(folder, filename)
fig.write_html(file_path, include_plotlyjs='cdn', full_html=True)
display(IFrame(src=file_path, width='100%', height='550'))

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display

# === Toggle Render Mode: 'vscode' or 'iframe'
render_mode = 'vscode'

if render_mode == 'vscode':
    pio.renderers.default = 'vscode'
else:
    pio.renderers.default = 'iframe'

# === Parameters ===
cube_size = 2
cube_center = np.array([0, 0, 2.5])
lens_center = np.array([-4, 0, 0])
image_plane_x = -6.5
refractive_index = 1.5

# === Cube vertices ===
half = cube_size / 2
local_vertices = np.array([
    [-half, -half, -half], [ half, -half, -half],
    [ half,  half, -half], [-half,  half, -half],
    [-half, -half,  half], [ half, -half,  half],
    [ half,  half,  half], [-half,  half,  half],
])
vertices = local_vertices + cube_center
x, y, z = vertices.T

# === Faces
faces = np.array([
    [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
    [0, 1, 5], [0, 5, 4], [1, 2, 6], [1, 6, 5],
    [2, 3, 7], [2, 7, 6], [3, 0, 4], [3, 4, 7]
])
i, j, k = faces.T

# === Face colors
face_colors = ['lightgray'] * len(faces)
highlight_faces = {'orange': [10, 11], 'green': [6, 7]}
for idx in highlight_faces['orange']:
    face_colors[idx] = 'orange'
for idx in highlight_faces['green']:
    face_colors[idx] = 'green'

# === Refraction through lens
def refract_ray(P, L, n):
    incident = P - L
    incident = incident / np.linalg.norm(incident)
    normal = np.array([1.0, 0, 0])
    cos_i = -np.dot(normal, incident)
    sin_t2 = (1/n)**2 * (1 - cos_i**2)
    if sin_t2 > 1.0:
        return None
    cos_t = np.sqrt(1 - sin_t2)
    refracted = (1/n) * incident + (cos_t - (1/n) * cos_i) * normal
    return refracted

def trace_through_lens(P, L, image_plane_x, n):
    direction = refract_ray(P, L, n)
    if direction is None:
        return L
    t = (image_plane_x - L[0]) / direction[0]
    return L + t * direction

projected_points = np.array([trace_through_lens(P, lens_center, image_plane_x, refractive_index) for P in vertices])
proj_x, proj_y, proj_z = projected_points.T

# === Lens (Vertical 3D Torus-like Lens)
theta = np.linspace(0, 2 * np.pi, 50)
phi = np.linspace(0, np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
r_outer = 0.5
r_inner = 0.1

lens_x = lens_center[0] + r_inner * np.cos(theta)
lens_y = r_outer * np.sin(phi)
lens_z = r_outer * np.cos(phi)

lens_surface = go.Surface(
    x=lens_x,
    y=lens_y,
    z=lens_z,
    showscale=False,
    opacity=0.8,
    colorscale=[[0, 'purple'], [1, 'purple']],
    name='Lens'
)

# === Plot elements
image_mesh = go.Mesh3d(x=proj_x, y=proj_y, z=proj_z, i=i, j=j, k=k,
                       facecolor=face_colors, opacity=0.6, name='Projected Image')
object_mesh = go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k,
                        facecolor=face_colors, opacity=0.4, name='Block')
yy, zz = np.meshgrid(np.linspace(-3, 3, 2), np.linspace(-3, 3, 2))
xx = np.full_like(yy, image_plane_x)
image_plane = go.Surface(x=xx, y=yy, z=zz, showscale=False, opacity=0.3,
                         name='Image Plane', colorscale='gray')

# === Rays
rays = [go.Scatter3d(
    x=[vertices[i][0], lens_center[0], projected_points[i][0]],
    y=[vertices[i][1], lens_center[1], projected_points[i][1]],
    z=[vertices[i][2], lens_center[2], projected_points[i][2]],
    mode='lines',
    line=dict(color='gray', width=2, dash='dot'),
    showlegend=False
) for i in range(8)]

# === Labels
corner_labels = go.Scatter3d(x=x, y=y, z=z, mode='markers+text',
    marker=dict(size=2, color='red'), text=[f'v{i}' for i in range(8)],
    textposition='top center', name='Vertices')
image_points = go.Scatter3d(
    x=proj_x, y=proj_y, z=proj_z,
    mode='markers+text',
    marker=dict(size=2, color='black'),
    text=[f'p{i}' for i in range(8)],
    textposition='bottom center',
    textfont=dict(size=8),
    name='Projected Corners'
)

# === Optical Axis and World/Camera Axes
optical_axis = go.Scatter3d(x=[image_plane_x - 2, cube_center[0] + 3],
    y=[0, 0], z=[0, 0], mode='lines',
    line=dict(color='black', width=4, dash='dash'), name='Optical Axis')

axis_len = 2
world_axes = [
    go.Scatter3d(x=[cube_center[0], cube_center[0] + axis_len], y=[cube_center[1], cube_center[1]], z=[cube_center[2], cube_center[2]], mode='lines', line=dict(color='red', width=5), name='World X'),
    go.Scatter3d(x=[cube_center[0], cube_center[0]], y=[cube_center[1], cube_center[1] + axis_len], z=[cube_center[2], cube_center[2]], mode='lines', line=dict(color='green', width=5), name='World Y'),
    go.Scatter3d(x=[cube_center[0], cube_center[0]], y=[cube_center[1], cube_center[1]], z=[cube_center[2], cube_center[2] + axis_len], mode='lines', line=dict(color='blue', width=5), name='World Z')
]
camera_axes = [
    go.Scatter3d(x=[lens_center[0], lens_center[0] + axis_len], y=[lens_center[1], lens_center[1]], z=[lens_center[2], lens_center[2]], mode='lines', line=dict(color='red', width=5), name="X′ (Camera)"),
    go.Scatter3d(x=[lens_center[0], lens_center[0]], y=[lens_center[1], lens_center[1] + axis_len], z=[lens_center[2], lens_center[2]], mode='lines', line=dict(color='green', width=5), name="Y′ (Camera)"),
    go.Scatter3d(x=[lens_center[0], lens_center[0]], y=[lens_center[1], lens_center[1]], z=[lens_center[2], lens_center[2] + axis_len], mode='lines', line=dict(color='blue', width=5), name="Z′ (Camera)")
]

# === Final Figure ===
fig = go.Figure(data=[
    object_mesh, image_mesh, image_plane,
    corner_labels, image_points, lens_surface, optical_axis,
    *world_axes, *camera_axes, *rays
])
fig.update_layout(
    title='Perspective Projection through a Vertical 3D Lens',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        camera=dict(
            eye=dict(x=1.5, y=1.2, z=1.0)
        )
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=0, r=0, t=40, b=40),
    showlegend=True
)

# === Save and Show ===
if render_mode == 'iframe':
    folder = "iframe_figures"
    filename = "vertical_lens_projection.html"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, filename)
    fig.write_html(file_path, include_plotlyjs='cdn', full_html=True)
    display(IFrame(src=file_path, width='100%', height='600'))
else:
    fig.show()